## Observations and Insights 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
study_mouse_df = study_results.join(mouse_metadata.set_index("Mouse ID"), on="Mouse ID")

# Display the data table for preview
study_mouse_df.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.0,0,Capomulin,Female,9,22
1,f932,0,45.0,0,Ketapril,Male,15,29
2,g107,0,45.0,0,Ketapril,Female,2,29
3,a457,0,45.0,0,Ketapril,Female,11,30
4,c819,0,45.0,0,Ketapril,Male,21,25


In [2]:
study_mouse_df

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,f932,0,45.000000,0,Ketapril,Male,15,29
2,g107,0,45.000000,0,Ketapril,Female,2,29
3,a457,0,45.000000,0,Ketapril,Female,11,30
4,c819,0,45.000000,0,Ketapril,Male,21,25
...,...,...,...,...,...,...,...,...
1888,r944,45,41.581521,2,Capomulin,Male,12,25
1889,u364,45,31.023923,3,Capomulin,Male,18,17
1890,p438,45,61.433892,1,Ceftamin,Female,11,26
1891,x773,45,58.634971,4,Placebo,Female,21,30


In [3]:
# Checking the number of mice.
mices = study_mouse_df['Mouse ID'].nunique()
print(f'We have a total of {mices} mice')

We have a total of 249 mice


In [4]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
study_mouse_df = study_mouse_df.sort_values(by=['Mouse ID','Timepoint'])
mice_dup = pd.DataFrame(study_mouse_df, columns=['Mouse ID', 'Timepoint'])
mice_dup = mice_dup[mice_dup.duplicated('Mouse ID')]
mice_dup.head()

,Mouse ID,Timepoint
389,a203,5
598,a203,10
758,a203,15
1018,a203,20
1295,a203,25


In [5]:
# Optional: Get all the data for the duplicate mouse ID. 
study_mouse_df.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
113,a203,0,45.000000,0,Infubinol,Female,20,23
389,a203,5,48.508468,0,Infubinol,Female,20,23
598,a203,10,51.852437,1,Infubinol,Female,20,23
758,a203,15,52.777870,1,Infubinol,Female,20,23
1018,a203,20,55.173336,1,Infubinol,Female,20,23


In [6]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
only_mice_df = pd.DataFrame(mice_dup, columns={'Mouse ID'})
clean_mice_df = only_mice_df.drop_duplicates()
clean_mice_df.head()

,Mouse ID
389,a203
456,a251
471,a262
250,a275
331,a366


In [7]:
# Checking the number of mice in the clean DataFrame.
clean_mice_df.reset_index(drop=True)

,Mouse ID
0,a203
1,a251
2,a262
3,a275
4,a366
...,...
232,z435
233,z578
234,z581
235,z795


## Summary Statistics

In [28]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
regimen_df = study_mouse_df.groupby(["Drug Regimen"])

# mean, median, variance, standard deviation, and SEM of the tumor volume.
mean_reg = regimen_df['Tumor Volume (mm3)'].mean()
median_reg = regimen_df['Tumor Volume (mm3)'].median()
var_reg = regimen_df['Tumor Volume (mm3)'].var()
std_reg = regimen_df['Tumor Volume (mm3)'].std()
sem_reg = regimen_df['Tumor Volume (mm3)'].sem()

# Assemble the resulting series into a single summary dataframe.
stat_regimen = pd.DataFrame({"Mean_Tumor_Vol_mm3":mean_reg,
                             "Median_Tumor_Vol_mm3":median_reg,
                             "Variance_Tumor_Vol_mm3":var_reg,
                             "STD_Tumor_Vol_mm3":std_reg,
                             "SEM_Tumor_Vol_mm3":sem_reg})
stat_regimen


,Mean_Tumor_Vol_mm3,Median_Tumor_Vol_mm3,Variance_Tumor_Vol_mm3,STD_Tumor_Vol_mm3,SEM_Tumor_Vol_mm3
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.322552,50.854632,42.351070,6.507770,0.512884
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


In [8]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line


## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pandas.



In [10]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
